In [27]:
import pandas as pd

In [28]:
df = pd.read_csv('../notion_data/S&P 500.csv')

In [29]:
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,12/31/2024,"5,881.63","5,919.74","5,929.74","5,868.86",NaN,-0.43%
1,12/30/2024,"5,906.94","5,920.67","5,940.79","5,869.16",NaN,-1.07%
2,12/27/2024,"5,970.84","6,006.17","6,006.17","5,932.95",NaN,-1.11%
3,12/26/2024,"6,037.59","6,024.97","6,049.75","6,007.37",NaN,-0.04%
4,12/24/2024,"6,040.04","5,984.63","6,040.10","5,981.44",NaN,1.10%


In [30]:
df = df[['Date', 'Price']]
df

,Date,Price
0,12/31/2024,"5,881.63"
1,12/30/2024,"5,906.94"
2,12/27/2024,"5,970.84"
3,12/26/2024,"6,037.59"
4,12/24/2024,"6,040.04"
...,...,...
2512,01/07/2015,"2,025.90"
2513,01/06/2015,"2,002.60"
2514,01/05/2015,"2,020.60"
2515,01/02/2015,"2,058.20"


In [31]:
df = df.rename(columns={'Date' : '날짜', 'Price' : '원자료'})

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      2517 non-null   object
 1   원자료     2517 non-null   object
dtypes: object(2)
memory usage: 39.5+ KB


In [33]:
df['날짜'] = pd.to_datetime(df['날짜'])

In [34]:
df = df.sort_values(by=['날짜'], ascending=True)

In [35]:
df = df.reset_index(drop=True)

df

,날짜,원자료
0,2014-12-31,"2,058.90"
1,2015-01-02,"2,058.20"
2,2015-01-05,"2,020.60"
3,2015-01-06,"2,002.60"
4,2015-01-07,"2,025.90"
...,...,...
2512,2024-12-24,"6,040.04"
2513,2024-12-26,"6,037.59"
2514,2024-12-27,"5,970.84"
2515,2024-12-30,"5,906.94"


In [36]:
# df = df.rename(columns/={'변환' : '날짜'})

In [37]:
import pandas as pd

# df['날짜'] = pd.to_datetime(df['날짜'])
df = df.sort_values('날짜').reset_index(drop=True)

# '날짜'를 인덱스로 설정
df = df.set_index('날짜')

# ,가 포함된 숫자 문자열을 정리하고 float으로 변환
df['원자료'] = df['원자료'].astype(str).str.replace(',', '').astype(float)

# 일별 수익률 계산
returns = df.pct_change()

# 결과 저장용 DataFrame 생성
result = pd.DataFrame(index=df.index)

# 앞으로 볼 윈도우 끝(일)과 라벨 정의: (5일→1~5, 10일→6~10, …)
windows = [
    (5, '(1~5)'),
    (10, '(6~10)'),
    (15, '(11~15)'),
    (20, '(16~20)')
]

# 각 컬럼에 대해 미래 5일씩 네 구간별 표준편차 계산
for col in df.columns:
    for end, label in windows:
        result[f"{col}{label}"] = (
            returns[col]
            .shift(-end)           # 미래 end일만큼 앞으로(음수) 시프트
            .rolling(window=5)     # 5일 윈도우
            .std()                 # 표준편차
        )

# 인덱스를 '날짜' 컬럼으로 되돌리고
result = result.reset_index()

print(result.head())


          날짜  원자료(1~5)  원자료(6~10)  원자료(11~15)  원자료(16~20)
0 2014-12-31       NaN        NaN         NaN         NaN
1 2015-01-02       NaN        NaN         NaN         NaN
2 2015-01-05       NaN        NaN         NaN         NaN
3 2015-01-06       NaN        NaN         NaN         NaN
4 2015-01-07  0.011015   0.009895     0.01007    0.012068


In [38]:
result.head(30)

,날짜,원자료(1~5),원자료(6~10),원자료(11~15),원자료(16~20)
0,2014-12-31,NaN,NaN,NaN,NaN
1,2015-01-02,NaN,NaN,NaN,NaN
2,2015-01-05,NaN,NaN,NaN,NaN
3,2015-01-06,NaN,NaN,NaN,NaN
4,2015-01-07,0.011015,0.009895,0.010070,0.012068
5,2015-01-08,0.002678,0.008596,0.010858,0.009069
6,2015-01-09,0.009221,0.007530,0.013469,0.009059
7,2015-01-12,0.008750,0.010854,0.014082,0.007914
8,2015-01-13,0.008931,0.012148,0.011971,0.007331
9,2015-01-14,0.009895,0.010070,0.012068,0.007174


In [39]:
result = result.rename(columns={
    '원자료(1~5)' : 'S&P500수익률표준편차(1~5)',
    '원자료(6~10)' : 'S&P500수익률표준편차(6~10)',
    '원자료(11~15)' : 'S&P500수익률표준편차(11~15)',
    '원자료(16~20)' : 'S&P500수익률표준편차(16~20)',    
})

In [40]:
result.head(30)

,날짜,S&P500수익률표준편차(1~5),S&P500수익률표준편차(6~10),S&P500수익률표준편차(11~15),S&P500수익률표준편차(16~20)
0,2014-12-31,NaN,NaN,NaN,NaN
1,2015-01-02,NaN,NaN,NaN,NaN
2,2015-01-05,NaN,NaN,NaN,NaN
3,2015-01-06,NaN,NaN,NaN,NaN
4,2015-01-07,0.011015,0.009895,0.010070,0.012068
5,2015-01-08,0.002678,0.008596,0.010858,0.009069
6,2015-01-09,0.009221,0.007530,0.013469,0.009059
7,2015-01-12,0.008750,0.010854,0.014082,0.007914
8,2015-01-13,0.008931,0.012148,0.011971,0.007331
9,2015-01-14,0.009895,0.010070,0.012068,0.007174


In [48]:
macro_df = pd.read_csv('../data/macro_data.csv')

In [49]:
macro_df.columns

Index(['회사명', 'stock_code', '상장일', 'kospi200(-20)', 'nasdaq(-20)',
       'putcall(-20)', 'VIXCLS', '미국10년물국채금리수익률(-20)', '시장금리국고채3년',
       '환율변동률(-20)',
       ...
       '코스닥수익률표준편차(-11~-15)', '코스닥수익률표준편차(-16~-20)', '코스피수익률표준편차(1~5)',
       '코스피수익률표준편차(6~10)', '코스피수익률표준편차(11~15)', '코스피수익률표준편차(16~20)',
       '나스닥수익률표준편차(1~5)', '나스닥수익률표준편차(6~10)', '나스닥수익률표준편차(11~15)',
       '나스닥수익률표준편차(16~20)'],
      dtype='object', length=168)

In [50]:
# 1) Convert both columns to datetime
macro_df['상장일'] = pd.to_datetime(macro_df['상장일'])
result['날짜'] = pd.to_datetime(result['날짜'])

# 2) Sort both DataFrames by their date keys
merged_full = macro_df.sort_values('상장일').reset_index(drop=True)
result = result.sort_values('날짜').reset_index(drop=True)

merged_full = pd.merge_asof(
    merged_full,
    result,
    left_on='상장일',
    right_on='날짜',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['날짜'])

In [51]:
merged_full.columns

Index(['회사명', 'stock_code', '상장일', 'kospi200(-20)', 'nasdaq(-20)',
       'putcall(-20)', 'VIXCLS', '미국10년물국채금리수익률(-20)', '시장금리국고채3년',
       '환율변동률(-20)',
       ...
       '코스피수익률표준편차(11~15)', '코스피수익률표준편차(16~20)', '나스닥수익률표준편차(1~5)',
       '나스닥수익률표준편차(6~10)', '나스닥수익률표준편차(11~15)', '나스닥수익률표준편차(16~20)',
       'S&P500수익률표준편차(1~5)', 'S&P500수익률표준편차(6~10)', 'S&P500수익률표준편차(11~15)',
       'S&P500수익률표준편차(16~20)'],
      dtype='object', length=172)

In [52]:
merged_full.to_csv('macro_data.csv', encoding='utf-8-sig', index=False)
